In [ ]:
import time
from tqdm import tqdm
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, ChiSqSelector, Normalizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml import Pipeline

# Set Up Spark Session
spark = SparkSession.builder.appName("TextClassificationWithProgress").getOrCreate()

# Load Data
textDF = spark.read.json("hdfs:///user/dic24_shared/amazon-reviews/full/reviews_devset.json")
textDF.createOrReplaceTempView("review")
reviewTextDF = spark.sql("SELECT category, reviewText FROM review")


# Preprocessing Pipeline Functions
# Pipeline Step 1: Preprocessing and Tokenization
regexTokenizer = RegexTokenizer().setInputCol("reviewText").setOutputCol("terms").setPattern("[\\s_\\*+-:,;+=/\\\\%€[0-9]$&§@#~`\"'\\.()\\t\\[\\]{}!?]")

# Pipeline Step 2: Stopword Removal
remover = StopWordsRemover().setInputCol("terms").setOutputCol("filteredTerms")

# Pipeline Step 3: Count Vectorization
countVec = CountVectorizer().setInputCol("filteredTerms").setOutputCol("features").setVocabSize(20000).setMinDF(5)

# Pipeline Step 4: TF-IDF Calculation
idf = IDF().setInputCol("features").setOutputCol("weightedFeatures")

# Pipeline Step 5: Category Encoding
encoder = StringIndexer().setInputCol("category").setOutputCol("categoryIndex")

# Pipeline Step 6: Chi-Square Selection
selector = ChiSqSelector().setNumTopFeatures(2000).setFeaturesCol("weightedFeatures").setLabelCol("categoryIndex").setOutputCol("selectedFeatures")

# Defining the Pipeline here
pipeline = Pipeline(stages=[
        regexTokenizer,
        remover,
        countVec,
        idf,
        encoder,
        selector
    ])


model = pipeline.fit(reviewTextDF)


result = model.transform(reviewTextDF)


# Show the resulting DataFrame with selected features
result.select("selectedFeatures").show(truncate=False)

# Optionally save the selected terms to a file
selected_terms = result.select("selectedFeatures").collect()

with open("output_ds.txt", "w") as output_file:
    for term in selected_terms:
        output_file.write(str(term) + "\n")



SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
24/05/28 09:07:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/28 09:07:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/28 09:07:04 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/05/28 09:07:04 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/05/28 09:07:04 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/05/28 09:07:0

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------